# LightAutoML

Это библиотека для автоматического машинного обучения (AutoML), которая предназначена для упрощения процесса построения и оптимизации моделей машинного обучения. Она позволяет пользователям создавать модели с минимальными усилиями и без глубоких знаний в области машинного обучения.

Кроме того, библиотека позволяет автоматически строить модели машинного обучения для различных задач, например для бинарной классификации, мультиклассовой классификации и регрессии, на табличных данных (в том числе вещественных и категориальных данных, датах, текстах и даже изображениях). Этот инструмент позволяет сосредоточиться на важных задачах машинного обучения — генерации признаков, выборе метрик, определении целевой переменной и так далее...

Как работает LightAutoML и какие ключевые функции она предоставляет:

1. Автоматический выбор моделей: LightAutoML автоматически выбирает подходящие модели машинного обучения для задачи, основываясь на данных. Она включает в себя несколько популярных алгоритмов и может настраивать их параметры.

2. Предобработка данных: Библиотека автоматически выполняет предварительную обработку данных, такую как заполнение пропущенных значений, нормализация, кодирование категориальных признаков и другие преобразования, необходимые для подготовки данных к обучению модели.

3. Автоматическая настройка гиперпараметров: LightAutoML использует методы оптимизации гиперпараметров, такие как случайный поиск или сеточный поиск, для настройки параметров моделей, чтобы достичь наилучших результатов.

4. Оценка моделей: Она предоставляет механизмы для оценки производительности различных моделей, включая использование кросс-валидации и метрик качества, таких как точность, F1-оценка и другие.

5. Поддержка различных типов задач: LightAutoML поддерживает множество типов задач машинного обучения, включая классификацию, регрессию и другие задачи. Она может работать как с табличными данными, так и с текстовыми и временными рядами.

Интерфейс: Библиотека предоставляет удобный интерфейс для работы с данными и моделями, что позволяет пользователям легко интегрировать её в свои проекты.

# Пример использования:

## Скачать библиотеку можно с помощью команды:
``` python
    pip install -U lightautoml
```

In [1]:
# Standard python libraries
import os
import time

# Essential DS libraries
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
import torch

# LightAutoML presets, task and report generation
from lightautoml.automl.presets.tabular_presets import TabularAutoML
from lightautoml.tasks import Task

'nlp' extra dependecy package 'gensim' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.
'nlp' extra dependecy package 'nltk' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.
'nlp' extra dependecy package 'transformers' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.
'nlp' extra dependecy package 'gensim' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.
'nlp' extra dependecy package 'nltk' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.
'nlp' extra dependecy package 'transformers' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/lightautoml/ml_algo/dl_model.py:42: UserWarning: 'transformers' - package isn't installed
  warnings.warn("'transformers' - package isn't installed")
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/lightautoml/text/embed.py:22: UserWarning: 'transformers' - package isn't installed
  warnings.warn("'transformers' - package isn't installed")
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/lightautoml/text/dl_transformers.py:25: UserWarning: 'transformers' - package isn't installed
  warnings.warn("'transformers' - package isn't installed")


Далее определяем константы (количество потоков, таймаут, имя целевой колонки, размер валидационной выборки и так далее).

In [2]:
N_THREADS = 4
N_FOLDS = 5
RANDOM_STATE = 42
TEST_SIZE = 0.2
TIMEOUT = 60 # equal to 10 minutes
TARGET_NAME = 'final_price'

Для максимальной воспроизводимости решения используем команды:

In [3]:
np.random.seed(RANDOM_STATE)
torch.set_num_threads(N_THREADS)

Указываем путь к папке с данными:

In [4]:
INPUT_DIR = 'Data/'

Загружаем тренировочный датасет (в нём будет 15 колонок):

In [5]:
train_data = pd.read_csv(INPUT_DIR + 'train_data.csv')
print(train_data.shape)
train_data.head()

(35000, 15)


,row_ID,vehicle_manufacturer,vehicle_model,vehicle_category,current_mileage,vehicle_year,vehicle_gearbox_type,doors_cnt,wheels,vehicle_color,vehicle_interior_color,car_vin,car_leather_interior,deal_type,final_price
0,0,TOYOTA,Aqua s,Sedan,133000,2014,Automatic,4/5,Right-hand drive,Silver,Black,NaN,0,For Sale,3650.0
1,1,MERCEDES-BENZ,C 220,Sedan,24500,2010,Manual,4/5,Left wheel,Silver,Black,NaN,0,For Sale,6800.0
2,2,HYUNDAI,Veloster,Hatchback,31000,2016,Tiptronic,2/3,Left wheel,Silver,Black,KMHTC6AE3GU293912,1,For Sale,6300.0
3,3,HYUNDAI,Santa FE,Jeep,115459,2015,Automatic,4/5,Left wheel,Blue,Black,NaN,1,For Sale,14488.0
4,4,TOYOTA,CHR,Jeep,18950,2019,Automatic,4/5,Left wheel,Black,NaN,JTNKHMBX7K1030253,1,For Sale,5000.0


Загружаем тестовый датасет (в нём будет 14 колонок, без колонки с целевой переменной):

In [6]:
test_data = pd.read_csv(INPUT_DIR + 'test_data.csv')
print(test_data.shape)
test_data.head()

(10697, 14)


,row_ID,vehicle_manufacturer,vehicle_model,vehicle_category,current_mileage,vehicle_year,vehicle_gearbox_type,doors_cnt,wheels,vehicle_color,vehicle_interior_color,car_vin,car_leather_interior,deal_type
0,35000,TOYOTA,Prius,Hatchback,323733,2012,Automatic,4/5,Left wheel,Grey,Black,JTDKN3DU6C5439638,1,For Sale
1,35001,HYUNDAI,Elantra,Sedan,112000,2013,Tiptronic,4/5,Left wheel,Grey,Black,SURATSHIA,1,For Sale
2,35002,LEXUS,NX 300,Jeep,16920,2018,Automatic,NaN,Left wheel,Brown,NaN,JTJYARBZ5J2104521,1,For Sale
3,35003,LEXUS,CT 200h,Hatchback,302742,2012,Automatic,4/5,Left wheel,White,NaN,JTHKD5BH4C2070945,1,For Sale
4,35004,TOYOTA,RAV 4,Jeep,1800,2002,Manual,4/5,Left wheel,Silver,Black,NaN,0,For Sale


Загружаем шаблон сабмита (по количеству строк совпадает с тестовыми данными):

In [7]:
submission = pd.read_csv(INPUT_DIR + 'sample_submission.csv')
print(submission.shape)
submission.head()

(10697, 2)


,row_ID,final_price
0,35000,0
1,35001,0
2,35002,0
3,35003,0
4,35004,0


Далее в бейзлайне идёт небольшая заготовка для генерации дополнительных признаков.

In [8]:
%%time

def create_expert_feats(data):
    pass

create_expert_feats(train_data)
create_expert_feats(test_data)

CPU times: user 17 µs, sys: 1e+03 ns, total: 18 µs
Wall time: 21 µs


Последний пункт подготовительного этапа нашей работы — разбиение на тренировочную (80 %, tr_data) и валидационную (20 %, te_data) выборки. 

In [9]:
tr_data, te_data = train_test_split(
    train_data, 
    test_size=TEST_SIZE,
    random_state=RANDOM_STATE
)

display(f'Data splitted. Parts sizes: tr_data = {tr_data.shape}, te_data = {te_data.shape}')

tr_data.head()

'Data splitted. Parts sizes: tr_data = (28000, 15), te_data = (7000, 15)'

,row_ID,vehicle_manufacturer,vehicle_model,vehicle_category,current_mileage,vehicle_year,vehicle_gearbox_type,doors_cnt,wheels,vehicle_color,vehicle_interior_color,car_vin,car_leather_interior,deal_type,final_price
18788,18788,HYUNDAI,Santa FE,Jeep,177719,2010,Automatic,4/5,Left wheel,Silver,Black,NaN,1,For Sale,7464.0
776,776,HYUNDAI,Santa FE,Jeep,143166,2011,Automatic,4/5,Left wheel,Grey,Black,NaN,1,For Sale,5959.0
480,480,BMW,X5 XDRIVE35 D,Jeep,149000,2015,Tiptronic,4/5,Left wheel,Black,Black,NaN,1,For Sale,23900.0
30078,30078,CHRYSLER,300,Sedan,133904,2014,Automatic,4/5,Left wheel,Silver,Black,NaN,1,For Sale,1750.0
29115,29115,TOYOTA,4Runner,Jeep,0,2018,Automatic,4/5,Left wheel,Black,Black,NaN,1,For Sale,100.0


28 000 строк отойдут в тренировочную выборку, 7 000 — в валидационную.

Первым делом нам необходимо определить, какую именно задачу нужно решить. Для этого в библиотеке LAMA используется объект TASK, который мы определили на подготовительном этапе.

In [10]:
task = Task('reg', loss = 'mse', metric = 'mse')

Далее необходимо указать роли переменных: сказать библиотеке, какая колонка является целевой. Кроме того, укажем на то, что колонку row_ID можно опустить.

In [11]:
roles = {
    'target': TARGET_NAME,
    'drop': ['row_ID']
}

И, наконец, создаём модель LightAutoML. Мы сделаем это при помощи класса TabularAutoML.

Класс TabularAutoML обладает следующими параметрами: 

* task — тип задачи (обязательный);
* timeout — ограничение по времени для тренировки;
* cpu_limit — ограничение по объёму CPU;
* reader_params — параметр объекта Reader, регулирующий чтение данных, например n_jobs, random_state.

Примечание: Мы не делали никаких манипуляций с датасетами: не очищали данные от пропусков, не удаляли дубликаты и так далее. LAMA это делает за нас!

Следующий этап — обучение тренировочной выборки.

Для этого используем метод fit_predict. 

Этот метод принимает три аргумента: 

* train_data — датасет для обучения (тренировочная выборка);
* roles — роли переменных (мы их уже определили);
* verbose — уровень детальности логирования (чем больше число, тем детальнее).

### Создание модели

In [12]:
automl = TabularAutoML(
    task = task, 
    timeout = TIMEOUT,
    cpu_limit = N_THREADS,
    reader_params = {'n_jobs': N_THREADS, 'cv': N_FOLDS, 'random_state': RANDOM_STATE}
)

### Тренировка модели

In [14]:
%%time

oof_pred = automl.fit_predict(tr_data, roles = roles, verbose = 1)

[23:08:33] Stdout logging level is INFO.
[23:08:33] Copying TaskTimer may affect the parent PipelineTimer, so copy will create new unlimited TaskTimer
[23:08:33] Task: reg

[23:08:33] Start automl preset with listed constraints:
[23:08:33] - time: 60.00 seconds
[23:08:33] - CPU: 4 cores
[23:08:33] - memory: 16 GB

[23:08:33] Train data shape: (28000, 15)

[23:08:44] Layer 1 train process start. Time left 48.95 secs
[23:08:45] Start fitting Lvl_0_Pipe_0_Mod_0_LinearL2 ...
[23:08:47] Time limit exceeded after calculating fold 1

[23:08:47] Fitting Lvl_0_Pipe_0_Mod_0_LinearL2 finished. score = -6300012352.069129
[23:08:47] Lvl_0_Pipe_0_Mod_0_LinearL2 fitting and predicting completed
[23:08:47] Time left 45.58 secs

[23:08:47] Selector_LightGBM fitting and predicting completed
[23:08:49] Start fitting Lvl_0_Pipe_1_Mod_0_LightGBM ...


В выводе на экран вы увидите подробную информацию о процессе построения модели, её описание вместе с out-of-fold-предсказанием (предсказанием на весь train_data, переменная oof_pred). При предсказании LAMA использует кросс-валидацию.

Можно дополнительно прописать вывод окончательного описания модели:

In [ ]:
display(automl.create_model_str_desc())

Следующий этап — обучение валидационной выборки. 

Для этого необходимо выполнить функцию automl_predict, передав ей на вход валидационные данные (te_data). Результаты передаются в переменную te_pred.

In [ ]:
%%time

te_pred = automl.predict(te_data)
display(f'Prediction for te_data:\n{te_pred}\nShape = {te_pred.shape}')

Измеряем качество модели, сравнивая MAE на тренировочной и валидационной выборках:

In [ ]:
display(f'TRAIN out-of-fold score: {mean_absolute_error(tr_data[TARGET_NAME].values, oof_pred.data[:, 0])}')
display(f'HOLDOUT score: {mean_absolute_error(te_data[TARGET_NAME].values, te_pred.data[:, 0])}')

Чтобы узнать, какие признаки повлияли на полученные предсказания, внутри библиотеки LAMA реализовано несколько методов, например fast и accurate. Эти методы помогают оценить, насколько хорошо или плохо влияет тот или иной признак на построение модели. 

Метод fast вызовем от автообученной модели automl, передавая на вход параметр fast.

In [ ]:
%%time

# Fast feature importances calculation
fast_fi = automl.get_feature_scores('fast')
fast_fi.set_index('Feature')['Importance'].plot.bar(figsize = (30, 10), grid = True)

Видно, что наибольшее влияние на стоимость оказывает год производства автомобиля. Далее идут пробег, количество дверей, наименование модели и так далее. 

Проведём оценку при помощи метода accurate.

In [ ]:
%%time

# Accurate feature importances calculation (Permutation importances) -  can take long time to calculate on bigger datasets
accurate_fi = automl.get_feature_scores('accurate', te_data, silent = False)

Отобразим на графике:

In [ ]:
accurate_fi.set_index('Feature')['Importance'].plot.bar(figsize = (30, 10), grid = True)

Порядок признаков поменялся, при этом достаточно важными остались и модель автомобиля, и год производства, и тип коробки передач. 

Судя по этим оценкам, результаты модели коррелируют со здравым смыслом. 

Следующий этап — предсказание на тестовых данных. Для этого снова запускаем функцию automl.predict, передав ей тестовый набор данных test_data.

In [ ]:
test_pred = automl.predict(test_data)
display(f'Prediction for te_data:\n{test_pred}\nShape = {test_pred.shape}')

Готовим сабмит, не сохраняя индекс:

In [ ]:
submission[TARGET_NAME] = test_pred.data[:, 0]
submission.to_csv('my_submission.csv', index = False)

### Результат

In [ ]:
result = pd.read_csv('Data/my_submission.csv')
display(result)